# Proyecto de Investigación: Modelos Numéricos
---

In [28]:
import ipywidgets as widgets
import numpy as np
from numpy import linalg as LA

## Modelo Matemático en Ecuaciones Diferenciales

$ m(x) \frac{\partial^{2}u}{\partial t^{2}} + EJ \frac{\partial^{4}u(x, t)}{\partial x^{4}} = p(x)g(t) \quad\text{;}\quad \Omega = \{x \in \mathbb{R}: 0 \leq x \leq L \}$

Condiciones de borde:

$\forall t \quad\text{en}\quad x = 0
\begin{cases}
u(0, t) = 0 \\
\frac{\partial u}{\partial x}|_{(0,t)} = 0
\end{cases}
$

$\forall t \quad\text{en}\quad x = L
\begin{cases}
EJ \frac{\partial^{2}u(x, t)}{\partial x^{2}} = M_{L} g_{c}(t) \\
EJ \frac{\partial^{3}u(x, t)}{\partial x^{3}} = 0
\end{cases}
$

Condiciones iniciales nulas.

## Datos del sistema

In [29]:
L = 420
EJ = 9.45e11
m = 64.8
p = 100
ML = 50000

## Modelo Matemático Discreto de N Grados de Libertad

Discretización del dominio

In [39]:
widgetN = widgets.FloatText()
display(widgetN)
N = 4
h = L/N

FloatText(value=0.0)

Forma discreta de la ecuación diferencial, de la forma:

$\mathbb{M} \cdot \bar{\ddot{u(t)}} + \mathbb{K} \cdot \bar{u(t)} = g(t) \cdot \bar{f_{p}} + g_{c}(t) \cdot \bar{f_{M}}$

Donde:

In [40]:
M = np.zeros((N, N))

for i in range(0, N):
    M[i, i] = m
    
K = np.zeros((N, N))
K[0, 0:3] = [7, -4, 1]
K[1, 0:4] = [-4, 6, -4, 1]

# Completar para N puntos de discretización
if (N > 4):
    for i in range(2, N-2):
        K[i, i-2:i+3] = [1, -4, 6, -4, 1]

K[N-2, N-4:N] = [1, -4, 5, -2]
K[N-1, N-3:N] = [2, -4, 2]

K = (EJ/h**4) * K

fp = np.full((N,), p)

fM = np.zeros((N,))
fM[N-2] = -1
fM[N-1] = 2
fM = (ML/(h*h)) * fM[:]

## Modelo Matemático de 1 Grado de Libertad

Se realiza un cambio de base del espacio generado por los modos naturales de vibración del modelo matemátiico discreto de N grados de libertad.

Es un problema de autovalores y autovectores, de la forma:

$(\mathbb{K} - \omega^{2}\mathbb{M}) \cdot \bar{\phi} = \bar{0}$

Por lo tanto, es necesario calcular las raíces del polinomio característico:

$det(\mathbb{K} - \omega^{2}\mathbb{M}) = 0$

In [66]:
alpha, v = LA.eig(LA.inv(M).dot(K))
print(alpha[0])
print(alpha[N-1])

1528.5734827879257
5.234918284189235


---
# Problema de Autovalores y Autovectores
Uno de los primeros problemas que surgen en el análisis es el cálculo de autovalores y autovectores. Por lo tanto, a continuación se implementan distintos métodos numéricos para su cómputo. El problema general se plante como:

$( \mathbb{A} - \lambda \cdot \mathbb{I} ) \cdot \bar{\phi} = 0$

## Método de potencias
### Cálculo de autovalor y autovector mayor
De Chapra "Métodos Numéricos para Ingenieros", página 810. Hay que tener en cuenta que para algunos casos especiales el método converge al segundo autovalor mayor.

In [64]:
A = LA.inv(M).dot(K)

lmbd = N*[1]
w = 1

i = 0;
while (1):
    newLmbd = A.dot(lmbd)
    newWmax = max(newLmbd)
    newLmbd = (1/newWmax) * newLmbd
    err = abs((newWmax - w)/newWmax)
    print(err)
    if (err < 0.001 or i > 1000):
        lmbd = newLmbd
        w = newWmax
        print("Error: " + str(err) + " - Iteración: " + str(i))
        break;
    lmbd = newLmbd
    w = newWmax
    i += 1
    
print(lmbd)
print(w)

0.997916275
0.5
0.20000000000000015
0.10614525139664785
0.061064094148537224
0.03321625809351938
0.016981466867421
0.008352575708242937
0.004025096981848794
0.0019200571002630522
0.0009114147481460174
Error: 0.0009114147481460174 - Iteración: 10
[ 1.         -1.20935664  0.89814786 -0.55931103]
1527.3221659476048


### Cálculo de autovalor menor
De Chapra "Métodos Numéricos para Ingenieros", página 812.

In [65]:
A = LA.inv(M).dot(K)

B = LA.inv(A)
lmbd = N*[1]
w = 1

i = 0;
while (1):
    newLmbd = B.dot(lmbd)
    newWmax = max(newLmbd)
    newLmbd = (1/newWmax) * newLmbd
    err = abs((newWmax - w)/newWmax)
    print(err)
    if (err < 0.001 or i > 1000):
        lmbd = newLmbd
        w = newWmax
        print("Error: " + str(err) + " - Iteración: " + str(i))
        break;
    lmbd = newLmbd
    w = newWmax
    i += 1
    
print(1/w)

2.528748360014233
0.4605180037902711
0.015131640778178067
0.0005765928636814348
Error: 0.0005765928636814348 - Iteración: 3
5.234802579069481


### Determinación de autovalores intermedios
Después de encontrar el mayor de los valores propios, es posible determinar los siguientes más grandes remplazando la
matriz original por una que incluya sólo los valores propios restantes. El proceso de eliminar el valor propio mayor conocido se llama **deflación**. Una de estas técnicas es el *método de Hotelling* que está diseñada para matrices simétricas, pero dado que la matriz A del problema no es simétrica queda descartado.

In [71]:
np.array_equal(A, A.transpose())

False

## Otros Métodos para la determinación de autovalores

Los siguientes métodos consisten en llevar la matriz **simétrica** A a una forma más sencilla:

- Método de Jacobi
- Método de Given
- Método de Householder

Nuevamente por ser para matrices simétricas se descartan.

Para el caso deseado, donde se requiere el cálculo de todos los autovalores de una matriz genérica, se dispone de el *método LR de Rutishauser* y el *método QR de Francis*.

De Chapra "Métodos Numéricos para Ingenieros", página 814, los métodos de Given y de Householder también se aplican a sistemas no simétricos. El resultado no será tridiagonal, sino más bien un tipo especial llamado *forma de Hessenberg*. Un procedimiento es aprovechar la velocidad del método de Householder para transformar la matriz a esta forma y, después, usar el algoritmo estable QR para hallar los valores propios. 